In [2]:
from utils import calculate_alignment_scores, calculate_novelty_scores, to_the_forms, calculate_f_scores
import os
import pandas as pd
import json

In [22]:
file = r"C:\Users\Valentin\Desktop\data_for_impact_analysis\data\aq\outputs\selt.txt_llama_t0.3_cyc1.csv"
df = pd.read_csv(file)
df = df.drop(["Reasoning", "Cycle"], axis=1)
df.to_clipboard(excel=True)

In [8]:

PROJECT_PATH = r"C:\Users\Valentin\Desktop\data_for_impact_analysis\data\aq"

CHANGE_REQUESTS_PATH = os.path.join(PROJECT_PATH, "change_requests.txt")
OUTPUTS_DIR = os.path.join(PROJECT_PATH, "outputs")
TEAM_DATA = os.path.join(PROJECT_PATH, "team_data")
relevancy = 0.46


def calculate_all_team():
    results = {}
    # List all output files
    output_files = [os.path.join(OUTPUTS_DIR,r"selt.txt_llama_t0.3_cyc1.csv")]
    # List all participant files (exclude _survey)
    participant_files = [f for f in os.listdir(TEAM_DATA) if not f.endswith('_survey.txt') and f.endswith('.txt')]

    for output_file in output_files:
        output_path = os.path.join(OUTPUTS_DIR, output_file)
        df1 = pd.read_csv(output_path)
        df1['Relevant']= 'Yes'
        participant_scores = {}
        alig_scores_list = []
        avg_align_scores = []
        avg_nov_scores = []
        avg_f_scores = []
        for participant_file in participant_files:
            participant_path = os.path.join(TEAM_DATA, participant_file)
            df2 = pd.read_csv(participant_path)
            scores, avg_score = calculate_alignment_scores(CHANGE_REQUESTS_PATH, df1, df2)
            scores2, avg_score2 = calculate_novelty_scores(df1,df2)
            scores3, avg_score3, _ = calculate_f_scores(CHANGE_REQUESTS_PATH,df1,df2)
            avg_score2  = avg_score2 * relevancy
            participant_name = participant_file.replace('.txt', '')
            participant_scores[participant_name] = (round(avg_score,2), round(avg_score2,2), round(avg_score3,2),scores )
            avg_align_scores.append(avg_score)
            avg_nov_scores.append(avg_score2)
            avg_f_scores.append(avg_score3)

        # Add average for the team
        participant_scores['average_alignment'] = round(sum(avg_align_scores) / len(avg_align_scores),2) if avg_align_scores else 0
        participant_scores['average_novelty'] = round(sum(avg_nov_scores) / len(avg_nov_scores),2) if avg_nov_scores else 0
        participant_scores['average_f_score'] = round(sum(avg_f_scores) / len(avg_f_scores),2) if avg_f_scores else 0


        results[output_file] = participant_scores
    return results

all_scores = calculate_all_team()
print(json.dumps(all_scores, indent=4))

{
    "C:\\Users\\Valentin\\Desktop\\data_for_impact_analysis\\data\\aq\\outputs\\selt.txt_llama_t0.3_cyc1.csv": {
        "combined": [
            0.74,
            0.14,
            0.69,
            [
                0.75,
                0.6666666666666666,
                0.6666666666666666,
                1.0,
                1.0,
                0.5,
                0.375,
                1.0
            ]
        ],
        "orange_nikita": [
            0.76,
            0.25,
            0.57,
            [
                1.0,
                0.0,
                0.6666666666666666,
                1.0,
                1.0,
                0.4,
                1.0,
                1.0
            ]
        ],
        "orange_sasha": [
            0.93,
            0.27,
            0.55,
            [
                1.0,
                0.8,
                1.0,
                1.0,
                1.0,
                1.0,
                0.6666666666666666,
            

In [1]:
from scipy.stats import mannwhitneyu

# Data (flattened lists)
senior = [
    0.4, 0.3333333333333333, 1.0, 0.4, 0.3333333333333333, 1.0, 1.0, 0.0,  # Optics
    0.3333333333333333, 0.75, 0.8, 0.4, 0.8333333333333334, 0.3, 0.5, 0.5, 0.6666666666666666, 0.5, 0.8, 1.0, 1.0, 0.8, 0.2, 0.5, 0.5, 0.75,  # Sensors
    1.0, 0.8, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0  # Orange
]

middle = [
    0.4, 0.4, 0.5, 1.0, 0.16666666666666666, 0.4, 0.6666666666666666, 0.0, 0.25, 0.5, 1.0, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 1.0, 0.0,  # Optics
    0.5, 0.75, 1.0, 1.0, 0.75, 0.25, 0.5714285714285714, 0.5, 0.6666666666666666,  # Sensors
    1.0, 0.5, 1.0, 1.0, 1.0, 0.5, 0.4, 1.0  # Orange
]

junior = [
    0.5, 0.4, 0.5, 0.6666666666666666, 0.0, 1.0, 1.0, 0.0,  # Optics
    1.0, 0.0, 0.6666666666666666, 1.0, 1.0, 0.4, 1.0, 1.0  # Orange
]

# Perform Mann-Whitney U tests
# Senior > Middle?
stat_senior_middle, p_senior_middle = mannwhitneyu(senior, middle, alternative='greater')
print(f"Senior > Middle: p-value = {p_senior_middle:.4f}")
if p_senior_middle < 0.05:
    print("✅ Statistically significant (Senior > Middle)")
else:
    print("❌ Not statistically significant")

# Middle > Junior?
stat_middle_junior, p_middle_junior = mannwhitneyu(middle, junior, alternative='greater')
print(f"\nMiddle > Junior: p-value = {p_middle_junior:.4f}")
if p_middle_junior < 0.05:
    print("✅ Statistically significant (Middle > Junior)")
else:
    print("❌ Not statistically significant")

Senior > Middle: p-value = 0.2292
❌ Not statistically significant

Middle > Junior: p-value = 0.6328
❌ Not statistically significant


In [2]:
import numpy as np
from scipy.stats import mannwhitneyu

# Raw data (manually parsed)
senior_scores = [
    0.4, 0.3333333333333333, 1.0, 0.4, 0.3333333333333333, 1.0, 1.0, 0.0,  # Optics
    0.3333333333333333, 0.75, 0.8, 0.4, 0.8333333333333334, 0.3, 0.5, 0.5, 0.6666666666666666,  # Sensors
    0.5, 0.8, 1.0, 1.0, 0.8, 0.2, 0.5, 0.5, 0.75,  # Sensors (continued)
    1.0, 0.8, 1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 1.0  # Orange
]

middle_scores = [
    0.4, 0.4, 0.5, 1.0, 0.16666666666666666, 0.4, 0.6666666666666666, 0.0,  # Optics
    0.25, 0.5, 1.0, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 1.0, 0.0,  # Optics (continued)
    0.5, 0.75, 1.0, 1.0, 0.75, 0.25, 0.5714285714285714, 0.5, 0.6666666666666666,  # Sensors
    1.0, 0.5, 1.0, 1.0, 1.0, 0.5, 0.4, 1.0  # Orange
]

junior_scores = [
    0.5, 0.4, 0.5, 0.6666666666666666, 0.0, 1.0, 1.0, 0.0,  # Optics
    1.0, 0.0, 0.6666666666666666, 1.0, 1.0, 0.4, 1.0, 1.0  # Orange
]

# Normalize scores by team average
def normalize_scores(scores):
    team_avg = np.mean(scores)
    return [x / team_avg for x in scores]

senior_normalized = normalize_scores(senior_scores)
middle_normalized = normalize_scores(middle_scores)
junior_normalized = normalize_scores(junior_scores)

# Perform Mann-Whitney U test (one-tailed: greater)
def test_greater(group1, group2, group1_name, group2_name):
    stat, p = mannwhitneyu(group1, group2, alternative='greater')
    print(f"{group1_name} > {group2_name}: p-value = {p:.4f}")
    if p < 0.05:
        print(f"✅ SIGNIFICANT: {group1_name} scores are higher than {group2_name}")
    else:
        print(f"❌ NOT SIGNIFICANT: No evidence that {group1_name} > {group2_name}")
    print()

# Test senior > middle > junior
test_greater(senior_normalized, middle_normalized, "Senior", "Middle")
test_greater(middle_normalized, junior_normalized, "Middle", "Junior")

Senior > Middle: p-value = 0.6636
❌ NOT SIGNIFICANT: No evidence that Senior > Middle

Middle > Junior: p-value = 0.2358
❌ NOT SIGNIFICANT: No evidence that Middle > Junior



In [31]:
import os
import pandas as pd

def clean_component_name(name):
    """Standardize component names for better deduplication"""
    return str(name).strip().lower()

def combine_and_deduplicate(team_data_folder):
    # Get all participant files (excluding survey files)
    participant_files = [
        f for f in os.listdir(team_data_folder) 
        if not f.endswith('_survey.txt') and f.endswith('.txt')
    ]
    
    # Initialize empty DataFrame
    combined_df = pd.DataFrame()
    
    # Process each file
    for file in participant_files:
        file_path = os.path.join(team_data_folder, file)
        try:
            # Read file (adjust separator if needed)
            df = pd.read_csv(file_path, sep=',')
            
            # Standardize column names
            
            
            # Ensure required columns exist
            if {'Component', 'Change'}.issubset(df.columns):
                combined_df = pd.concat([combined_df, df], ignore_index=True)
        except Exception as e:
            print(f"Error processing {file}: {e}")
    
    if combined_df.empty:
        return pd.DataFrame()
    
    # Clean component names for consistent comparison
    combined_df['component_clean'] = combined_df['Component'].apply(clean_component_name)
    
    # Drop duplicates based on cleaned component name and change value
    # Keep first occurrence, case insensitive
    deduplicated_df = combined_df.drop_duplicates(
        subset=['component_clean', 'Change'],
        keep='first'
    ).drop(columns=['component_clean'])
    
    # Reset index after deduplication
    deduplicated_df.reset_index(drop=True, inplace=True)
    deduplicated_df.groupby("Change")
    
    return deduplicated_df
TEAM_DATA = r"C:\Users\Valentin\Desktop\data_for_impact_analysis\data\inTouch\team_data"
# Usage example:
result_df = combine_and_deduplicate(TEAM_DATA)

# Display results
print(f"Found {len(result_df)} unique components after deduplication")
result_df.to_csv(os.path.join(TEAM_DATA, "combined.txt"),index= False)

Found 128 unique components after deduplication


In [ ]:
df1 = pd.read_csv(r'')
to_the_forms(df1, change_requests_path, outputs)

In [6]:
#df1 - LLMs response, df2 - human response
df1 = pd.read_csv(r'C:\Users\Valentin\Desktop\data_for_impact_analysis\data\aq\outputs\selt.txt_llama_t0.3_cyc1.csv')
df2 = pd.read_csv(r'C:\Users\Valentin\Desktop\data_for_impact_analysis\data\aq\team_data\orange_valya.txt')
df1['Relevant'] = "Yes"
relevancy = 0.9

In [16]:
participant_files = [f for f in os.listdir(TEAM_DATA) if f.endswith('_survey.txt') and f.endswith('.txt')]

participant_relevancy_score  = {}
avg = []


for file in participant_files:
    file = os.path.join(TEAM_DATA, file)
    df_n = pd.read_csv(filepath_or_buffer=file,sep = '\t', header = None)
    df_n = df_n.drop(df_n.columns[[0,1]], axis=1).T  # Drop columns at positions 0, 1
    df_n.columns = ['Relevant']
    relevancy = len(df_n[df_n["Relevant"] == "Yes"])/len(df_n)
    participant_relevancy_score[os.path.basename(file)] = relevancy
    avg.append(relevancy)

avg = sum(avg)/len(avg)
print(json.dumps(participant_relevancy_score, indent=4))
print(f"/n average: {avg}")

{
    "Angel_survey.txt": 0.5416666666666666,
    "EgorK_survey.txt": 0.3333333333333333,
    "IlonaP_survey.txt": 0.2708333333333333,
    "Ivan_survey.txt": 0.5833333333333334
}
/n average: 0.43229166666666663


In [8]:
scores, avg_score = calculate_alignment_scores(change_requests_path,df1, df2)
print("avg alignment:", round(avg_score,2))

avg alignment: 0.55


In [71]:
# df_n = pd.read_csv(filepath_or_buffer=r'team_data\nastya_survey.csv',sep = '\t', header = None)
# df_n = df_n.drop(df_n.columns[[0,1]], axis=1).T  # Drop columns at positions 0, 1
# df_n.columns = ['Relevant']
# df_n.index = range(0, len(df_n))

In [72]:
novelty_scores, nov = calculate_novelty_scores(df1, df2)
nov = nov*relevancy 
print("Novelty with relevancy adjustment", round(nov,2) )

Novelty with relevancy adjustment 0.14
